# Agent prototype: nist_xps_db aggregation

Agent to aggregate binding energies from the NIST XPS database, combine the data into a histogram, and update the nist_xps_db dataset entry with the histogram.

In [1]:
import json
import pandas as pd
from io import StringIO
import matplotlib as plt
import globus_auth
from gmeta_utils import gmeta_pop, format_gmeta

%matplotlib inline

In [2]:
client = globus_auth.login("https://search.api.globus.org/", "mdf")

# Fetch and aggregate records

In [9]:
count = 0
num_processed = 0
data_list = []
while True:
    query = {
        "q": ("source_name:nist_xps_db AND node_type:record AND version:0.2.0 AND "
        "scroll_id:(>=" + str(count) + " AND <" + str(count + 10000) + ")"),
        "advanced": True,
        "limit": 10000
    }
    raw_res = client.structured_search(query)
    search_res = gmeta_pop(raw_res, True)
    for res in search_res:
        data_dict = json.loads(res.get("mdf-raw", "{}"))
        if data_dict.get("Binding Energy (eV)", None):
            data_list.append({
                "composition": res["mdf-material_composition"],
                "binding_energy": float(data_dict.get("Binding Energy (eV)", "nan"))
    #            "excitation_energy": float(data_dict.get("Excitation Energy", "nan"))
            })
    num_ret = len(search_res)
    if num_ret:
        num_processed += num_ret
        count += 10000
    else:
        break
print(len(data_list), "/", num_processed, "|", len(data_list) - num_processed)
bind_en_list = [d["binding_energy"] for d in data_list]
print("Max:", max(bind_en_list), "\nMin:", min(bind_en_list))

KeyError: 'mdf-material_composition'

In [4]:
df = pd.DataFrame(data_list)
buckets = list(range(0, 4000, 100))
df['bucket'] = pd.cut(df['binding_energy'], buckets, labels=buckets[:-1])
hist = df.sort_values("bucket").groupby("bucket").bucket.count()

In [5]:
df2 = pd.DataFrame(hist)
df2.columns = ["count"]
df2.reset_index(inplace=True)
df2.columns = ["binding energy (eV)", "count"]
data = df2.as_matrix().tolist()
headers = ["binding energy (eV)", "count"]

# Update dataset entry

In [6]:
query = {
    "q": "source_name:nist_xps_db AND node_type:dataset",
    "advanced": True
}
raw_res = client.structured_search(query)
search_res = gmeta_pop(raw_res)
if len(search_res) != 1:
    raise ValueError("Incorrect number of results: " + str(len(search_res)))
ingest = search_res[0]
ingest["globus_subject"] = raw_res["gmeta"][0]["subject"]
ingest["acl"] = ["public"]
ingest["http://materialsdatafacility.org/#histogram"] = {"http://materialsdatafacility.org/#headers": headers, "http://materialsdatafacility.org/#matrix": data}
#ingest["http://materialsdatafacility.org/#nist_xps_db:histogram"] = {"http://globus.org/#fieldthing": "otherthing"}
#ingest["http://materialsdatafacility.org/#nist_xps_db:histogram"] = "12345"
#ingest["https://google.com/#testingthing"] = "thingthing"
gmeta = format_gmeta([format_gmeta(ingest)])

gmeta = json.loads(json.dumps(gmeta).replace("mdf-publish.publication.community", "http://globus.org/publish-terms/#publication/community"))

In [7]:
client.ingest(gmeta)

GlobusHTTPResponse({'success': True})

# Check ingest

In [8]:
query = {
    "q": "source_name:nist_xps_db AND node_type:dataset",
    "advanced": True
}
raw_res = client.structured_search(query)
search_res = gmeta_pop(raw_res, True)
#print(json.dumps(search_res[0], sort_keys=True, indent=4, separators=(',', ': ')))

In [9]:
hist = search_res[0]["histogram"]
df2 = pd.DataFrame(hist["matrix"], columns=hist["headers"])

In [10]:
df2

,binding energy (eV),count
0,0,4563
1,100,5197
2,200,3970
3,300,2036
4,400,1590
5,500,3579
6,600,514
7,700,516
8,800,510
9,900,345
